In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
import plotly.graph_objects as go
import sqlite3 

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM,Dense, Dropout, BatchNormalization
from keras.preprocessing.sequence import TimeseriesGenerator

%matplotlib inline  




In [2]:
# #yf.download('GE',start='2019-06-01', interval='1d',  end='2021-06-01',progress=False)
# df = yf.download('GE',start='2016-06-01', interval='1d',  end='2021-07-13',progress=False)

# df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
# df["Date"] = df.index
# df.head()
 
#data.plot(figsize=(10,10))

In [3]:
# df.info()

In [2]:
database_path = "./stocktrx.db"
# Create your connection.
cnx = sqlite3.connect(database_path)
df = pd.read_sql_query("SELECT * FROM stocks", cnx)
df.head()

,TrxDate,Ticker,Open,High,Low,Close,AdjClose,Volume
0,2017-07-19 00:00:00,AAPL,37.619999,37.855000,37.487499,37.755001,35.945583,83692000.0
1,2017-07-19 00:00:00,BA,209.429993,210.929993,208.990005,210.880005,198.707397,2258400.0
2,2017-07-19 00:00:00,DAL,53.770000,54.340000,53.099998,53.790001,50.100189,10733000.0
3,2017-07-19 00:00:00,GE,25.817308,25.990385,25.807692,25.903847,24.509535,25784096.0
4,2017-07-19 00:00:00,GOOG,967.840027,973.039978,964.030029,970.890015,970.890015,1224500.0


In [3]:
#unique tickers from the df
# df["Ticker"].value_counts().count() #1000
tickers = df['Ticker'].unique()
for t in tickers:
    print(t)

AAPL
BA
DAL
GE
GOOG
IBM
MSFT


In [5]:
# total number of tickers - 7
# df["Ticker"].value_counts().count()
# df.columns
# df["Ticker"].value_counts() #1000 records for each stock
# total_rows = df["Close"].count()
# total_rows

# df.columns

In [4]:
# drop columns we wont be using
df = df.drop(columns=[ 'Open', 'High', 'Low', 'AdjClose', 'Volume'])
df.head()


,TrxDate,Ticker,Close
0,2017-07-19 00:00:00,AAPL,37.755001
1,2017-07-19 00:00:00,BA,210.880005
2,2017-07-19 00:00:00,DAL,53.790001
3,2017-07-19 00:00:00,GE,25.903847
4,2017-07-19 00:00:00,GOOG,970.890015


In [5]:
#GE
ge = df.loc[df["Ticker"] == 'GE']
ge.head()
tf = ge


In [6]:
# other stocks close and date
close_data = df['Close'].values
close_data = close_data.reshape((-1,1))

test_data = tf['Close'].values
test_data = test_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data
close_test = test_data

date_train = df['TrxDate']
date_test = tf['TrxDate']

print(len(close_train))
print(len(close_test))

7000
1000


In [7]:
look_back = 30 # looking back 30 days

# look_back = 4
train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=40)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [8]:
# Create the model
model = Sequential()
# add to model: LTSM algo
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# initialize the number of epochs we are going to train our data for
num_epochs = 100
# use fit_generator to train the data (may use .fit for 2.2.0 or higher versions of tf - works the same)
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)
# model.fit(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/100
175/175 [==============================] - 2s 6ms/step - loss: 375519.2559
Epoch 2/100
175/175 [==============================] - 1s 6ms/step - loss: 373284.2630
Epoch 3/100
175/175 [==============================] - 1s 6ms/step - loss: 322430.5579
Epoch 4/100
175/175 [==============================] - 1s 6ms/step - loss: 294765.9126
Epoch 5/100
175/175 [==============================] - 1s 6ms/step - loss: 234506.5587
Epoch 6/100
175/175 [==============================] - 1s 6ms/step - loss: 209741.3375
Epoch 7/100
175/175 [==============================] - 1s 6ms/step - loss: 239775.9808
Epoch 8/100
175/175 [==============================] - 1s 6ms/step - loss: 208613.5406
Epoch 9/100
175/175 [==============================] - 1s 6ms/step - loss: 221531.3728
Epoch 10/100
175/175 [==============================] - 1s 6ms/step - loss: 194142.1618
Epoch 11/100
175/175 [==============================] - 1s 6ms/step - loss: 187262.7991
Epoch 12/100
175/175 [===================

In [10]:
print(model)

In [11]:
#Test
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "GE Stock",
    xaxis = {'title' : "TrxDate"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace2, trace3], layout=layout)
fig.show()

In [95]:
# close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    # last_date = df['Date'].values[-1]
    last_date = df['TrxDate'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)
print(forecast_dates)

[Timestamp('2021-07-08 00:00:00', freq='D'), Timestamp('2021-07-09 00:00:00', freq='D'), Timestamp('2021-07-10 00:00:00', freq='D'), Timestamp('2021-07-11 00:00:00', freq='D'), Timestamp('2021-07-12 00:00:00', freq='D'), Timestamp('2021-07-13 00:00:00', freq='D'), Timestamp('2021-07-14 00:00:00', freq='D'), Timestamp('2021-07-15 00:00:00', freq='D'), Timestamp('2021-07-16 00:00:00', freq='D'), Timestamp('2021-07-17 00:00:00', freq='D'), Timestamp('2021-07-18 00:00:00', freq='D'), Timestamp('2021-07-19 00:00:00', freq='D'), Timestamp('2021-07-20 00:00:00', freq='D'), Timestamp('2021-07-21 00:00:00', freq='D'), Timestamp('2021-07-22 00:00:00', freq='D'), Timestamp('2021-07-23 00:00:00', freq='D'), Timestamp('2021-07-24 00:00:00', freq='D'), Timestamp('2021-07-25 00:00:00', freq='D'), Timestamp('2021-07-26 00:00:00', freq='D'), Timestamp('2021-07-27 00:00:00', freq='D'), Timestamp('2021-07-28 00:00:00', freq='D'), Timestamp('2021-07-29 00:00:00', freq='D'), Timestamp('2021-07-30 00:00:00'

In [12]:

def predict(num_prediction, model):
    prediction_list = test_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = tf['TrxDate'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)
print(forecast_dates)

[Timestamp('2021-07-08 00:00:00', freq='D'), Timestamp('2021-07-09 00:00:00', freq='D'), Timestamp('2021-07-10 00:00:00', freq='D'), Timestamp('2021-07-11 00:00:00', freq='D'), Timestamp('2021-07-12 00:00:00', freq='D'), Timestamp('2021-07-13 00:00:00', freq='D'), Timestamp('2021-07-14 00:00:00', freq='D'), Timestamp('2021-07-15 00:00:00', freq='D'), Timestamp('2021-07-16 00:00:00', freq='D'), Timestamp('2021-07-17 00:00:00', freq='D'), Timestamp('2021-07-18 00:00:00', freq='D'), Timestamp('2021-07-19 00:00:00', freq='D'), Timestamp('2021-07-20 00:00:00', freq='D'), Timestamp('2021-07-21 00:00:00', freq='D'), Timestamp('2021-07-22 00:00:00', freq='D'), Timestamp('2021-07-23 00:00:00', freq='D'), Timestamp('2021-07-24 00:00:00', freq='D'), Timestamp('2021-07-25 00:00:00', freq='D'), Timestamp('2021-07-26 00:00:00', freq='D'), Timestamp('2021-07-27 00:00:00', freq='D'), Timestamp('2021-07-28 00:00:00', freq='D'), Timestamp('2021-07-29 00:00:00', freq='D'), Timestamp('2021-07-30 00:00:00'

In [25]:
trace1 = go.Scatter(
    x = date_test,
    y = close_test,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = "GE Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
pfig = go.Figure(data=[trace1, trace2], layout=layout)
pfig.show()

In [ ]:
# close db connection
cnx.close()